In [6]:
# import

import csv
import numpy as np

In [7]:
# read csv files

with open('polygons.csv', newline='') as csvfile0:
    Poly = list(csv.reader(csvfile0))

with open('Wqueries.csv', newline='') as csvfile1:
    Wq = list(csv.reader(csvfile1))
    
with open('Dqueries.csv', newline='') as csvfile2:
    Dq = list(csv.reader(csvfile2))

In [12]:
# string to float

def strtofloat(string_data):
    return list(map(float, string_data))

# calculate MBR

def MBR(coordination):
    min_x, min_y = np.min(coordination, axis=0)
    max_x, max_y = np.max(coordination, axis=0)
    return [min_x, max_x, min_y, max_y]

# calculate global MBR

def globalMBR(coordset):
    xmin_MBR = []
    xmax_MBR = []
    ymin_MBR = []
    ymax_MBR = []

    for i in coordset:
        xmin_MBR.append(i[0])
        xmax_MBR.append(i[1])
        ymin_MBR.append(i[2])
        ymax_MBR.append(i[3])
    
    return [min(xmin_MBR), max(xmax_MBR), min(ymin_MBR), max(ymax_MBR)]

In [13]:
# calculate all the polygons

polygons = []

for i in Poly:
    newlist = []
    for j in i:
        newj = j.split(' ')
        newj_float = strtofloat(newj)
        newlist.append(newj_float)
    polygons.append(newlist)

# calculate all the polygons' MBRs

polygon_MBR = []

for i in polygons:
    theMBR = MBR(i)
    polygon_MBR.append(theMBR)
    
global_MBR = globalMBR(polygon_MBR)

In [16]:
print('first polygon: ' + str(polygons[0]))
print('first polygon\'s MBR: ' + str(polygon_MBR[0]))
print('the global MBR: ' + str(global_MBR))

first polygon: [[-87.866893, 32.825274], [-87.863009, 32.815742], [-87.863257, 32.815652], [-87.867142, 32.82521], [-87.866893, 32.825274]]
first polygon's MBR: [-87.867142, -87.863009, 32.815652, 32.825274]
the global MBR: [-179.147236, 178.591597, -14.548699, 71.359879]


In [17]:
# rectangular range queries

In [25]:
# range query function

def Windowquery(Wquery, recMBR):
    Wx_min = Wquery[0]
    Wx_max = Wquery[1]
    Wy_min = Wquery[2]
    Wy_max = Wquery[3]
    
    polygon_number = 0
    intersection_rec = []
    
    for i in polygon_MBR:
        xmin = i[0]
        xmax = i[1]
        ymin = i[2]
        ymax = i[3]
        
        if xmax< Wx_min or xmin > Wx_max or ymax < Wy_min or ymin > Wy_max:
            polygon_number+= 1
        else:
            intersection_rec.append(polygon_number)
            polygon_number+= 1
            
    return intersection_rec

# transfer Wqueries

Wqueries = []
for i in Wq:
    i_float = strtofloat(i)
    Wqueries.append(i_float)
    
# scan Wqueries

for q in Wqueries:
    res_rec = Windowquery(q, polygon_MBR)
    print('query:' + str(q) + ' has ' + str(len(res_rec)) + ' results in the filter step: ' + str(res_rec))
    # print(q)
    # print(Window_query(q, polygon_MBR))

query:[-88.0, -87.5, 32.5, 33.0] has 19 results in the filter step: [0, 1, 2, 3, 40, 43, 746, 749, 760, 1371, 1372, 1375, 1376, 2611, 2612, 2613, 2854, 2880, 2882]
query:[-88.0, -87.0, 32.0, 32.5] has 71 results in the filter step: [742, 743, 744, 745, 747, 748, 750, 752, 753, 754, 755, 756, 757, 758, 759, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 897, 901, 902, 903, 911, 913, 1351, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1363, 1364, 1365, 1366, 1367, 1368, 1373, 1374, 2304, 2306, 2307, 2310, 2311, 2312, 2314, 2315, 2319, 2320, 2322, 2325, 2326, 2327, 2333, 2334, 2340, 2576, 2734, 2786, 2927]
query:[-87.5, -87.0, 33.0, 34.0] has 76 results in the filter step: [446, 495, 496, 500, 644, 645, 652, 657, 658, 659, 660, 661, 662, 664, 665, 667, 671, 672, 675, 677, 688, 690, 692, 693, 695, 698, 700, 701, 702, 703, 704, 705, 706, 707, 709, 711, 713, 715, 718, 719, 721, 722, 725, 726, 727, 728, 729, 731, 732, 737, 738, 740, 741, 1016, 1017, 1020, 1022, 1023, 1024, 1029

In [26]:
# circle distance queries

In [34]:
# distance query function
def Distquery(Dquery, recMBR):
    R = Dquery[2]
    Xd = Dquery[0]
    Yd = Dquery[1]
    Q = Dquery[:2]
    
    polygon_number = 0
    intersection_circle = []
    
    for i in recMBR:
        X1 = i[0]
        Y1 = i[2]
        X2 = i[1]
        Y2 = i[3]
    
        Xn = max(X1, min(Xd, X2))
        Yn = max(Y1, min(Yd, Y2))
    
        Distx = Xn - Xd
        Disty = Yn - Yd
        
        if (Distx**2 + Disty**2) <= R**2:
            intersection_circle.append(polygon_number)
            polygon_number+= 1
        else:
            polygon_number+= 1
    
    return Q, R, intersection_circle 

# transfer Dqueries

Dqueries = []
for i in Dq:
    i_float = strtofloat(i)
    Dqueries.append(i_float)
    
# scan Dqueries

for q in Dqueries:
    Q, R, res_circle = Distquery(q, polygon_MBR)
    print('query:' + str(Q) + ' with dist ' + str(R) + ' has ' + str(len(res_circle)) + ' results in the filter step: ' + str(res_circle))


query:[-88.0, 32.0] with dist 0.2 has 10 results in the filter step: [745, 751, 758, 762, 1537, 1541, 1542, 1545, 1546, 1555]
query:[-88.0, 32.0] with dist 0.4 has 75 results in the filter step: [293, 294, 295, 742, 744, 745, 748, 750, 751, 753, 756, 758, 759, 761, 762, 764, 765, 766, 767, 769, 772, 1514, 1515, 1516, 1517, 1518, 1520, 1523, 1524, 1525, 1526, 1528, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1563, 1573, 1577, 1578, 1579, 2328, 2329, 2330, 2560, 2734, 2761, 2762, 2881, 2915]
query:[-88.0, 32.0] with dist 0.6 has 165 results in the filter step: [284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 321, 325, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 338, 351, 352, 520, 521, 522, 523, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772,